<a href="https://colab.research.google.com/github/karaage0703/stable-diffusion-colab-tools/blob/main/001_stable_diffusion_gui_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion GUI Basic
Stable Diffusion easy and useful GUI basic tool

GitHub repository is below:
[stable-diffusion-colab-tools](https://github.com/karaage0703/stable-diffusion-colab-tools)

In [ ]:
#@title **Hugging Face Login**
#@markdown　You need access token of Hugging Face.

!pip -qq install git+https://github.com/huggingface/diffusers.git
!pip -qq install transformers scipy ftfy gradio
!pip -qq install "ipywidgets>=7,<8"

from google.colab import output
output.enable_custom_widget_manager()

from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
#@title **Launch App**
#@markdown　Execute and click URL ex: `Running on public URL: https://xxxx.gradio.app` 

import gradio as gr
import torch
from diffusers import StableDiffusionPipeline

device = "cuda"
model_id = "CompVis/stable-diffusion-v1-4"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    revision="fp16",
    torch_dtype=torch.float16,
    use_auth_token=True,
).to(device)

# Full model
#pipe = StableDiffusionPipeline.from_pretrained(
#    model_id,
#    use_auth_token=True,
#).to(device)


def infer(prompt, num_images, num_inference_steps,):
    generator = torch.Generator(device=device)
    latents = None
    seeds = []

    width = 512
    height = 512

    num_images = int(num_images)
    num_inference_steps = int(num_inference_steps)

    for _ in range(num_images):
        # Get a new random seed, store it and use it as the generator state
        seed = generator.seed()
        seeds.append(seed)
        generator = generator.manual_seed(seed)
    
        image_latents = torch.randn(
            (1, pipe.unet.in_channels, height // 8, width // 8),
            generator = generator,
            device = device
        )
        latents = image_latents if latents is None else torch.cat((latents, image_latents))

    with torch.autocast('cuda'):
        images = pipe(
            [prompt] * num_images,
            guidance_scale=7.5,
            num_inference_steps=num_inference_steps,
            latents = latents,
        )['sample']

    return images

from IPython.display import clear_output

block = gr.Blocks(css=".container { max-width: 800px; margin: auto; }")

with block as demo:
    gr.Markdown("<h1><center>Stable Diffusion Tool</center></h1>")
    gr.Markdown(
        'Stable Diffusion useful web tool'
    )
    with gr.Group():
        with gr.Box():
            gr.Markdown(
                'Enter prompt and Run!!'
            )
            with gr.Row().style(mobile_collapse=False, equal_height=True):

                text = gr.Textbox(
                    label='Enter prompt', show_label=False, max_lines=1
                ).style(
                    border=(True, False, True, True),
                    rounded=(True, False, False, True),
                    container=False,
                )
                btn = gr.Button("Run").style(
                    margin=False,
                    rounded=(False, True, True, False),
                )

        num_images = gr.Number(
                    label='Enter number of images', value=1
                )

        num_inference_steps = gr.Number(
                    label='Enter number of inference steps', value=50
                )

        gallery = gr.Gallery(label="Generated images", show_label=False).style(
            grid=[2], height="auto"
        )
        text.submit(infer, inputs=[text, num_images, num_inference_steps], outputs=gallery)
        btn.click(infer, inputs=[text, num_images, num_inference_steps], outputs=gallery)

    gr.Markdown(
        """___
   <p style='text-align: center'>
   Created by CompVis and Stability AI
   <br/>
   </p>"""
    )

clear_output()
demo.launch(debug=False)